In [ ]:
# ONE-TIME Requirement Installation
!pip install pandas, O365

In [ ]:
# EDITABLE INFORMATION

# column names
name = 'First and Last Name'
phone = 'Phone number'
slots = 'Select at least one time slot'
level = 'Skill level'
email = 'Email'

# session info
valid_slots = [
    'Tuesday, 9th of Nov, 22-23h at Tennisstadion',
    'Thursday, 11th of Nov, 9-10am \(morning!\) at Tennisstadion',
    'Friday, 12th of Nov, 21-22h at TSK Malmen'
]

# available courts for each session
courts_num = [6, 1, 5] # same length as valid_slots

# file name of the email template
# The template is in HTML,
#     which can be edited using any online editor, eg:
#     https://html-online.com/editor/
email_template = 'template_email.html'

# excel file name
excel_fn = 'Tennis Session Sign-Up (Responses) (1).xlsx'

# outlook authentication info
outlook_sender = 'kthracketklubb@outlook.com' # replace it
client_id = 'replace it'
client_secret = 'replace it'

In [ ]:
with open(email_template, 'r') as template:
    email_template = template.read()

def format_email(
        session_time='Wednesday, October 6th, from 10- 11h',
        session_info='TSK Malmen (Södra Fiskartorpsvägen 20 114 33 Stockholm), it\'s right beside Tennisstadion. Door code if needed: 1949.',
        session_venue='TSK Malmen main lobby',
        meeting_time='10am',
        meeting_place='court D12',
    ):
    return email_template.format(
        session_time=session_time,
        session_venue=session_venue,
        session_info=session_info,
        meeting_time=meeting_time,
        meeting_place=meeting_place,
    )


In [ ]:
from O365 import Account

def send_message_receivers(receivers, message=format_email()):
    account = Account((client_id, client_secret))
    m = account.new_message()
    for r in receivers:
        m.to.add(r)
    m.subject = 'KTH Racketklubb CONFIRMATION'
    m.body = message
    m.send()
    print(f'Successfully sent to {receivers}')

In [ ]:
import pandas as pd
import numpy as np
import warnings
import functools
warnings.filterwarnings('ignore')

people_per_court = 4
level_scores = {
    'Beginner': 0,
    'Intermediate I': 1,
    'Intermediate II': 2,
    'Advanced/Pro': 3,
}
v = 0
tmp = {}
for i in valid_slots:
    tmp[i] = v
    v += 1
valid_slots = tmp

def clean_df(df):
    na_phone = df[df[phone].isnull()].index.values.tolist()
    na_email = df[df[email].isnull()].index.values.tolist()
    df_cleaned = df.dropna(subset=[phone, email])
    
    df_cleaned.loc[:,email] = df_cleaned[email].apply(lambda e: e.strip())
    df_cleaned.loc[:,phone] = df_cleaned[phone].apply(lambda e: str(e).strip())
    
    df_cleaned = df_cleaned.drop_duplicates(subset=[phone], keep='last')
    df_cleaned = df_cleaned.drop_duplicates(subset=[email], keep='last')
    nan_level_list = df_cleaned[df_cleaned[level].isnull()][name].tolist()
    df_cleaned = df_cleaned[~df_cleaned[slots].str.extract('|'.join(map(lambda s: f'({s})', list(valid_slots.keys())))).isnull().all(axis=1)]
    
    print(f'NaN levels will be replaced with "Intermediate I" for people: \n{nan_level_list}')
    df_cleaned[level].fillna('Intermediate I', inplace=True)
    df_cleaned.loc[:,level] = df_cleaned[level].map(level_scores)
    for k in valid_slots:
        df_cleaned.loc[:,slots] = df_cleaned[slots].str.replace(k, str(valid_slots[k]))
    df_cleaned.loc[:,slots] = df_cleaned[slots].map(lambda v: list(map(lambda vv: int(vv), v.split(', '))))
    df_cleaned.sort_values(level, inplace=True)
    
    df_cleaned = df_cleaned.reset_index(drop=True)
    print(f'Signed persons: {df_cleaned.shape[0]}')
    return df_cleaned

def get_key(val, d):
    for key, value in d.items():
        if val == value:
             return key

def flatten_ll(l):
    return [j for i in l for j in i]

In [ ]:
# Statistics
df = pd.read_excel(excel_fn)
df = df[[name, phone, slots, level, email]]
df = clean_df(df)

print()

for slot in valid_slots:
    size = np.sum(df[slots].map(lambda v: valid_slots[slot] in v))
    print(f'{size} persons chose "{slot}"')
    
idx_choice_level = {} # for each person, records of their choices and skill level
level_idx = {} # records of people in each skill level
for idx, row in df.iterrows():
    choices = row[slots]
    if choices != []:
        idx_choice_level[idx] = [choices, row[level]]
        if row[level] not in level_idx:
            level_idx[row[level]] = [idx,]
        else:
            level_idx[row[level]].append(idx)
print()
for level_name in level_scores:
    if level_scores[level_name] in level_idx:
        print(f'{level_name}: {len(level_idx[level_scores[level_name]])}')

print()

In [ ]:
idx_l = df.index.tolist()
i = 0
spots = [] # two dimensional array
# first index: session id; second index: court id; elements: indices of people

for i in courts_num:
    spots.append([[]] * i)

def append_to_spots(x, y, value):
    spot = spots[x][y].copy()
    spot.append(value)
    spots[x][y] = spot

got_spot_tracer = [False] * df.shape[0]
    
def calculate_weight(idx):
    if not got_spot_tracer[idx]:
        return 100
    if idx not in remained:
        return 0
    return len(remained[idx])
        
    
remained = {}
for idx, row in df.iterrows():
    remained[idx] = row[slots].copy()

# blue: tolerate difference in adjacent levels (except beginners)
# force: ignore difference in levels
# fill: ignore rules to fill all empty spots
def run_allocate(input_df, blur=False, debug=False, fill=False, force=False):
    got_iter = False
    for idx, row in input_df.iterrows():
        got = None
        for session_idx in remained[idx]:
            for court_idx in range(courts_num[session_idx]):
                if idx in spots[session_idx][court_idx]:
                    continue
                if blur and row[level] == 0: # a beginner
                    continue
                if len(spots[session_idx][court_idx]) == 0:
                    append_to_spots(session_idx, court_idx, idx)
                    got = session_idx
                    if debug:
                        print(f'{df.loc[idx][name]}{{{df.loc[idx][level]}}} got {got}{{{df.loc[spots[got][court_idx][0]][level]}}}')
                    break
                
                level_diff = df.loc[spots[session_idx][court_idx][0]][level] - row[level]
                max_diff = 1 if blur else 0
                if force:
                    max_diff = 10
                if fill and len(spots[session_idx][court_idx]) < people_per_court:
                    append_to_spots(session_idx, court_idx, idx)
                    got = session_idx
                    if debug:
                        print(f'{df.loc[idx][name]}{{{df.loc[idx][level]}}} got {got}{{{df.loc[spots[got][court_idx][0]][level]}}}')
                    break
                elif np.abs(level_diff) <= max_diff \
                        and len(spots[session_idx][court_idx]) < people_per_court:
                    append_to_spots(session_idx, court_idx, idx)
                    got = session_idx
                    if debug:
                        print(f'{df.loc[idx][name]}{{{df.loc[idx][level]}}} got {got}{{{df.loc[spots[got][court_idx][0]][level]}}}')
                    break
            if got is not None:
                break
        if got is not None:
            got_iter = True
            got_spot_tracer[idx] = True
            remained[idx].remove(got)
    return got_iter


In [ ]:
debug = False

print('Allocating people who signed only ONE session...')
v_choice = df[df[slots].map(lambda v: len(v) == 1)]
run_allocate(v_choice, debug=debug)
print()

print('Forcely allocating people who signed only ONE session...')
v_choice = df[df[slots].map(lambda v: len(v) == 1)]
run_allocate(v_choice, debug=debug, force=True)
print()

print('Allocating people who have NOT gotten a spot...')
while True:
    got_iter = False
    df['weight'] = df.apply(lambda row: calculate_weight(row.name), axis=1)
    # allocate one spot each time
    got_iter = run_allocate(df[df['weight']==100], debug=debug)
    if not got_iter:
        break
print()

print('BLUR allocating people who have NOT gotten a spot...')
# in this case, an Intermediate I player can be allocated to a Beginner court or a Intermediate II court
# EXCEPTION: a beginnner can only be in a Beginner court
while True:
    got_iter = False
    df['weight'] = df.apply(lambda row: calculate_weight(row.name), axis=1)
    # allocate one spot each time
    got_iter = run_allocate(df[df['weight']==100], blur=True, debug=debug)
    if not got_iter:
        break
print()

print('Forcely allocating people who have NOT gotten a spot...')
while True:
    got_iter = False
    df['weight'] = df.apply(lambda row: calculate_weight(row.name), axis=1)
    # allocate one spot each time
    got_iter = run_allocate(df[df['weight']==100], force=True, debug=debug)
    if not got_iter:
        break
print()

print('Allocating all signed spots...')
while True:
    got_iter = False
    df['weight'] = df.apply(lambda row: calculate_weight(row.name), axis=1)
    # allocate one spot each time
    got_iter = run_allocate(df.sort_values(by='weight', ascending=False), debug=debug)
    if not got_iter:
        break
print()

print('BLUR allocating adjacent levels for all people...')
while True:
    got_iter = False
    df['weight'] = df.apply(lambda row: calculate_weight(row.name), axis=1)
    # allocate one spot each time
    got_iter = run_allocate(df.sort_values(by='weight', ascending=False), blur=True, debug=debug)
    if not got_iter:
        break
print()

print('Filling courts...')
while True:
    got_iter = False
    df['weight'] = df.apply(lambda row: calculate_weight(row.name), axis=1)
    # allocate one spot each time
    got_iter = run_allocate(df.sort_values(by='weight', ascending=False), fill=True, debug=debug)
    if not got_iter:
        break
print()
        
print('Unmet for NAME{LEVEL}: UNMET_SPOTS out of SIGNED_SPOTS')
keys = list(remained.keys())
for idx in keys:
    if remained[idx] != []:
        row = df.loc[idx]
        print(f'Unmet for {df.loc[idx][name]}{{{df.loc[idx][level]}}}:\t{remained[idx]}\tout of\t{row[slots]}')
print()

In [ ]:
persons_per_session = []

# Display allocations
for session_info in valid_slots:
    i = valid_slots[session_info]
    print(f'{session_info}:')
    for l in spots[i]:
        print(f'Court for {get_key(df.loc[l[0]][level], level_scores)}')
        print('\t' + ', '.join([f"{row[name]}{{{row[level]}}}" for idx, row in df.loc[l].iterrows()]))
    print('==========================================')
    indices = flatten_ll(spots[i])
    persons_per_session.append(indices)

total_signup = functools.reduce(lambda a, b: a+b, [len(row[slots]) for idx, row in df.iterrows()])
print('Total unmet:', functools.reduce(lambda a, b: a+b, [len(remained[k]) for k in remained]), end=' ') # total count per person per session
print('out of', total_signup)
# print('Total count per person per session', functools.reduce(lambda a, b: a+b, [len(persons_per_session[i]) for i in range(len(persons_per_session))])) # total count per person per session)

In [ ]:
email_l = []

print()
# Get email list for each session
for session_info in valid_slots:
    i = valid_slots[session_info]
    print(f'{session_info}:')
    l = df.loc[persons_per_session[i]][email].values.tolist()
    email_l.append(l)
    for e in l:
        print(f'\t{e}')

In [ ]:
# Send emails in batches
# Replace the first parameter to email_l[0] for session 0
# Uncomment if you wanna send emails via code

# valid_slots = [
#     'Tuesday, 9th of Nov, 22-23h at Tennisstadion',
#     'Thursday, 11th of Nov, 9-10am \(morning!\) at Tennisstadion',
#     'Friday, 12th of Nov, 21-22h at TSK Malmen'
# # ]

send_message_receivers(
    email_l[0],
    format_email(
        session_time='Tuesday, 9th of Nov, 22-23h',
        session_info='Tennisstadion (Södra Fiskartorpsvägen 20 114 33 Stockholm).',
        meeting_place='court D14',
        meeting_time='22h'
    )
)



send_message_receivers(
    email_l[1],
    format_email(
        session_time='Thursday, 11th of Nov, 9-10am \(morning!\)',
        session_info='Tennisstadion (Södra Fiskartorpsvägen 20 114 33 Stockholm).',
        meeting_place='court A1',
        meeting_time='9am'
    )
)


send_message_receivers(
    email_l[2],
    format_email(
        session_time= 'Friday, 5th of Nov, 21-22h',
        session_info='TSK Malmen (Södra Fiskartorpsvägen 20 114 33 Stockholm), it\'s right beside Tennisstadion. Door code if needed: 1949.',
        meeting_place='TSK Malmen main lobby',
        meeting_time='21h'
    )
)


send_message_receivers(
    email_l[3],
    format_email(
        session_time='Sunday, 7th of Nov, 8-9am \(morning!\)',
        session_info='TSK Malmen (Södra Fiskartorpsvägen 20 114 33 Stockholm), it\'s right beside Tennisstadion. Door code if needed: 1949.',
        meeting_place='TSK Malmen main lobby',
        meeting_time='8am'
    )
)



In [ ]:
email_l[2]